## gs_SlidetoIcePipeline01_EffDet
- Using latest version of icevision.  Need to modify 'parser_mixins.py' to include 'ParserMixin' as public class in header of file.
- Included cell to do initial download of some slide. Uncomment after first download

- Config paramerters are contained in lib_pipeline_ice_config.py

- Install Order of WSI Dependencies:  
    1) OpenSlide: apt-get install openslide-tools  
    2) OpenSlide Python wrapper: pip install openslide-python  
    3) SlideRunner: pip install SlideRunner (SlideRunner may have additional dependencies, install as arise.)


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sqlite3
from SlideRunner.dataAccess.database import Database
from tqdm import tqdm
from pathlib import Path
import openslide
import time
import pdb

from random import randint

In [ ]:
from icevision.all import *
from fastai.callback.wandb import *
from fastai.callback.tracker import SaveModelCallback

In [ ]:
from lib_pipeline_ice_config import *
from lib_pipeline_ice_helper import *

## Download WSI

In [ ]:
# import urllib.request
# import os
# from tqdm import tqdm
# import requests

# downloadableFiles = {'WSI/MITOS_WSI_CCMCT_ODAEL.sqlite':
#                           'https://ndownloader.figshare.com/files/16261571?private_link=a82ddb634864c24f4aee' ,                  
#                      'WSI/9374efe6ac06388cc877.svs': # 26
#                           'https://ndownloader.figshare.com/files/16261553?private_link=a82ddb634864c24f4aee',
#                      'WSI/c3eb4b8382b470dd63a9.svs': # 4
#                           'https://ndownloader.figshare.com/files/16261466?private_link=a82ddb634864c24f4aee',
#                      'WSI/2e611073cff18d503cea.svs': # 32
#                           'https://ndownloader.figshare.com/files/16261439?private_link=a82ddb634864c24f4aee',
#                     }
# # Create folder for WSI if nonexistant
# if not os.path.isdir('WSI'):
#     os.mkdir('WSI')

# tqdm.write('Downloading all files from figshare - take a coffee and sit down, this may take awhile ...')  

# from time import sleep    
# sleep(0.5)
                 
# for fname in tqdm(list(downloadableFiles.keys())):
#     urllib.request.urlretrieve(downloadableFiles[fname],fname)


## Display Configuration Parameters (lib_pipeline_config.py)

In [ ]:
#Display Config Parameters (edit file)
with open('./lib_pipeline_ice_config.py') as f:
    print(f.read())

## Select Annotation Data from SlideRunner DB

In [ ]:
slide_count = 0
if slide_count > 0:
    getslides = "SELECT uid, filename FROM Slides LIMIT " + str(slide_count)
else:
    getslides = "SELECT uid, filename FROM Slides"
# print(getslides)

In [ ]:
database = Database()
print(str(pathDB/dbname))
database.open(pathDB/dbname)

In [ ]:
dir_slides = os.listdir(pathWSI)
database_slides = [s[1] for s in database.execute(getslides).fetchall()]
annotated_and_existing_slides = list(set(dir_slides) & set(database_slides))
slide_list = "', '".join(map(str, annotated_and_existing_slides))

train_slide_query = "SELECT uid, filename FROM Slides WHERE filename IN ('" + slide_list + "')"
# print(train_slide_query)

In [ ]:
files_train, lbl_bbox_train = get_slide_annotations(database, params['size'], train_slide_query)

## Randomize and Split Annotatation Data

In [ ]:
train_images = params['ds_size']
# val_images = int(params['ds_size']*0.2)
test_images = 16

train_files = list(np.random.choice(files_train, train_images))
# valid_files = list(np.random.choice(files_train, val_images))
test_files = list(np.random.choice(files_train, test_images))
print('train_files #:', len(train_files))
print('train_files wdith:', train_files[0].width)

In [ ]:
class_map = ClassMap(params['class_map'])

In [ ]:
parser = SlideContainerParser(train_files)
train_rs, valid_rs = parser.parse(autofix=True)

In [ ]:
# print(len(train_rs))
# print(len(valid_rs))
# show_records(train_rs[:6], ncols=3, class_map=class_map)

In [ ]:
# Transforms
train_tfms = tfms.A.Adapter([*tfms.A.aug_tfms(size=params['size'], presize=params['size_trans']), tfms.A.Normalize()])
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(params['size']), tfms.A.Normalize()])

In [ ]:
# Datasets
train_ds = Dataset(train_rs, train_tfms)
valid_ds = Dataset(valid_rs, valid_tfms)

In [ ]:
# samples = [train_ds[0] for _ in range(3)]
# show_samples(samples, ncols=3, class_map=class_map)

In [ ]:
# DataLoaders
train_dl = efficientdet.train_dl(train_ds, batch_size=params['batch_size'], num_workers=0, shuffle=True)
valid_dl = efficientdet.valid_dl(valid_ds, batch_size=params['batch_size'], num_workers=0, shuffle=False)

In [ ]:
train_dl.batch_size
valid_dl.batch_size

In [ ]:
# batch, samples = first(train_dl)
# show_samples(samples[:6], class_map=class_map, ncols=3)

## Model & Parameters Setup

In [ ]:
if params['size_progresize'] > 0:
    patchsize = str(params['size_progresize'])
else:
    patchsize = str(params['size'])

params['epochs1'] = 15
params['lr1'] =3e-3
params['epochs2'] = 10
params['lr2'] = 3e-5
params['wandb'] = False

params['Run-Notes'] = 'Progressive Resize = False; Running multiples of 256 with 500, 2000, 5000 patches; Finding best EffDet backbone'
params.update(wandb_run_name = params['modelclass'] + '-' + params['model'] + '_' + str(len(params['class_map'])-1) +'cls_' + patchsize + 'px_' + str(params['ds_size']) + 'ds_' + str(params['run']) +'_'+ params['run_datetime'])
params.update(model_save_name = params['modelclass'] + '-' + params['model'] + '_' + str(len(params['class_map'])-1) +'cls_' + patchsize + 'px_' + str(params['ds_size']) + 'ds_' + str(params['run']) +'_'+ params['run_datetime'])

In [ ]:
#EfficentDet
#Backbones - Edit model in Config file
# tf_efficientdet_lite0, efficientdet_d0, efficientdet_d1, efficientdet_d2, efficientdet_d3, efficientdet_d4, 
# efficientdet_d5, efficientdet_d6, efficientdet_d7, efficientdet_d7x

model = efficientdet.model(model_name=params['model'], 
                           num_classes=len(params['class_map']), 
                           img_size=params['size'])

In [ ]:
metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

In [ ]:
if (params['wandb']==True): 
    WANDB_NOTEBOOK_NAME = 'gs_SlidetoIcePipeline01_EffDet.ipynb'
    wandb.init(project=params['project'], name=params['wandb_run_name'], config=params)
    config = wandb.config

In [ ]:
if (params['wandb']==True): 
    callbacks = [WandbCallback(log_preds=True, log_dataset=False, log_model=False)]
else: 
    callbacks = []

In [ ]:
learn = efficientdet.fastai.learner(dls=[train_dl, valid_dl], cbs = callbacks, model=model, metrics=metrics)

## Model Training

In [ ]:
# learn.split([model.encoder[6], model.c5top5])
# learn.freeze_to(-2)

In [ ]:
learn.freeze()
# learn.lr_find()

In [ ]:
params.update(lr1 = 1e-3)
params.update (epochs1 = 15)

learn.fit_one_cycle(params['epochs1'], params['lr1'], cbs=SaveModelCallback())

In [ ]:
params.update(current_saved_model='stage-1-' + str(params['size']) + 'px-' + params['model'])
print(params['current_saved_model'])
learn.save(params['current_saved_model'])
# wandb.log(params)

In [ ]:
learn.freeze()
learn.lr_find()

In [ ]:
%matplotlib inline
params.update(lr1 = 7e-6)
params.update (epochs1 = 10)
# wandb.log({'lr1': params['lr1'], 'epochs': params['epochs1']})

learn.fit_one_cycle(params['epochs1'], params['lr1'])

In [ ]:
params.update(current_saved_model='stage-2-' + str(params['size']) + 'px-' + params['model'])
print(params['current_saved_model'])
learn.save(params['current_saved_model'])
# wandb.log(params)

## Review Model Run

In [ ]:
efficientdet.show_results(model, train_ds, class_map=class_map)

In [ ]:
valid_test_dl = efficientdet.infer_dl(valid_ds, batch_size=params['batch_size'])
samples, preds = efficientdet.predict_dl(model, valid_test_dl)

In [ ]:
# class_map = ClassMap(params['class_map'])

In [ ]:
if (params['wandb']==True): 
#     wandb_images = wandb_img_preds(samples, preds, class_map, add_ground_truth=True) 
#     wandb.log({"Validation Images Prediction-" + str(size_trans) + "px": wandb_images})
    wandb.join()

## Export Model/Weights

In [ ]:
model_folder = params['model_save_name']
path_model = Path(path/'models'/model_folder)
path_model_str = str(path_model) #needed for os folder creations

In [ ]:
if os.path.exists(path_model_str):
   shutil.rmtree(path_model_str)
if not os.path.exists(path_model_str):
    print(path_model_str)
    os.mkdir(path_model_str)

In [ ]:
#Save model weights
model_saved = params['model_save_name'] + '.pth'
# torch.save(model.state_dict(), data_dir/'mitosis_tf_efficientdet_lite0.pth')
torch.save(model.state_dict(), path_model/model_saved)

In [ ]:
print(path_model/model_saved)
params['size']

## Test Predictions from Exported Model

In [ ]:
model2 = efficientdet.model(model_name=params['model'], 
                           num_classes=len(params['class_map']), 
                           img_size=params['size'])
state_dict = torch.load(path_model/model_saved)
model2.load_state_dict(state_dict)

In [ ]:
# valid_test_dl = efficientdet.infer_dl(valid_ds, batch_size=params['batch_size'])
# samples, preds = efficientdet.predict_dl(model, valid_test_dl)

In [ ]:
efficientdet.show_results(model2, test_rs, class_map=class_map)